# Loading Data

In [20]:
import pandas as pd

# Read the data
data  = pd.read_csv("../Data/Combined.csv")

# Split the data into train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)
X_train = train['Tweets_clean_more']
y_train = train['Label']
X_test = test['Tweets_clean_more']
y_test = test['Label']

# Tokenization & Padding

In [21]:
from keras.preprocessing.text import Tokenizer

# Tokenize the data
tokenizer = Tokenizer(num_words=5000, oov_token="")
tokenizer.fit_on_texts(X_train)
X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_test_tokenized = tokenizer.texts_to_sequences(X_test)

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad the data
maxlen = 100
X_train = pad_sequences(X_train_tokenized, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test_tokenized, padding='post', maxlen=maxlen)

# Embedding Matrix - GloVe

In [23]:
import numpy as np

vocab_size = len(tokenizer.word_index) + 1

embedding_index = dict()
f = open("../Input Source/golve.twitter.27B/glove.twitter.27B.200d.txt")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embedding_index))

FileNotFoundError: [Errno 2] No such file or directory: '../Input Source/golve.twitter.27B/glove.twitter.27B.200d.txt'